In [1]:
# Install pyspark and findspark
!pip install --ignore-install -q pyspark
# Install findspark library
!pip install --ignore-install -q findspark

In [2]:
# Import findspark
import findspark
findspark.init()

In [3]:
import sys
sys.version_info
print(sys.version)

3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 11:57:02) [GCC 12.3.0]


### 1. Set up spark context and SparkSession

In [4]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark-DecisitonTreeClassifier_Iris") \
    .getOrCreate()

### 2. Load dataset

In [6]:
# Load the Iris dataset (assuming you have it in a CSV format)
iris_data = spark.read.csv("iris-data.csv", header=True, inferSchema=True)

In [7]:
# Let's assume that the "class" column is our target variable (label)
# and the other columns are our features
feature_cols = iris_data.columns[:-1]

In [8]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
# Convert string labels into numerical labels
indexer = StringIndexer(inputCol="class", outputCol="label")
iris_data = indexer.fit(iris_data).transform(iris_data)


In [9]:
# Create a feature vector by assembling the feature columns
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(iris_data)

In [10]:
# Split the data into training and testing sets
(training_data, testing_data) = data.randomSplit([0.8, 0.2], seed=123)

### 3. Create DecisionTree Classifier

In [11]:
# Create a DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=5, minInfoGain=0.001, impurity="entropy")

In [12]:
# Train the model
model = dt.fit(training_data)

In [13]:
# Make predictions on the testing data
predictions = model.transform(testing_data)

### 3. Evaluation

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.93


### 4. Feature importance

In [15]:
feature_importance = model.featureImportances.toArray()

# Show feature importance
for i, column in enumerate(assembler.getInputCols()):
    print(f"Feature '{column}': {feature_importance[i]:.2f}")

Feature 'sepal length': 0.03
Feature 'sepal width': 0.00
Feature 'petal length': 0.63
Feature 'petal width': 0.34


### 5. Visualize the Decision Tree

In [16]:
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_50e929847c47, depth=4, numNodes=15, numClasses=3, numFeatures=4
  If (feature 2 <= 2.45)
   Predict: 0.0
  Else (feature 2 > 2.45)
   If (feature 3 <= 1.75)
    If (feature 2 <= 4.95)
     If (feature 3 <= 1.65)
      Predict: 1.0
     Else (feature 3 > 1.65)
      Predict: 2.0
    Else (feature 2 > 4.95)
     If (feature 0 <= 6.35)
      Predict: 2.0
     Else (feature 0 > 6.35)
      Predict: 1.0
   Else (feature 3 > 1.75)
    If (feature 2 <= 4.85)
     If (feature 0 <= 5.95)
      Predict: 1.0
     Else (feature 0 > 5.95)
      Predict: 2.0
    Else (feature 2 > 4.85)
     Predict: 2.0



In [18]:
# Stop the Spark session
spark.stop()